#### *Better late than never* (решения в конце тетрадки).

In [1]:
import json, os
import pandas as pd
from nltk.corpus import stopwords
import numpy as np
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
morph = MorphAnalyzer()
stops = set(stopwords.words('russian'))

In [2]:
pd.set_option('display.max_colwidth', 1000)

## Данные

Возьмем данные вот отсюда - https://github.com/mannefedov/ru_kw_eval_datasets Там лежат 4 датасета (статьи с хабра, с Russia Today, Независимой газеты и научные статьи с Киберленинки). Датасет НГ самый маленький, поэтому возьмем его в качестве примера.

In [3]:
# скачаем данные в папке data и распакуем их
PATH_TO_DATA = '.\data'

In [4]:
files = [os.path.join(PATH_TO_DATA, file) for file in os.listdir(PATH_TO_DATA)]

Объединим файлы в один датасет.

In [5]:
data = pd.concat([pd.read_json(file, lines=True, encoding='utf8') for file in files][:1], axis=0, ignore_index=True)

In [6]:
data.shape

(988, 5)

In [7]:
data.head(3)

,content,keywords,summary,title,url
0,"В среду состоялось отложенное заседание Совета по федеральным государственным образовательным стандартам (ФГОС) при Министерстве образования и науки РФ. Собрание должно было состояться еще в понедельник, но было перенесено по просьбе членов совета. И вот пришло сообщение, что общественники выразили согласие с позицией министерства. Новые ФГОСы приняты.\nНа вчерашнем заседании был принят ФГОС по начальной общеобразовательной школе. До 28 марта продлятся косультации по ФГОСам для средней школы.\nНапомним, что накануне Гильдия словесников разместила открытое письмо на имя министра образования и науки РФ Ольги Васильевой. По мнению авторов письма, новые ФГОСы грубо нарушают права детей, уже проучившихся по существующему стандарту до 6-го класса. Приняв новый стандарт, Министерство образования дает право контролирующим органам ловить детей на незнании большого списка произведений (235 за пять лет обучения). «Это исключает возможность полноценного их освоения, создает риск формального, п...","[школа, образовательные стандарты, литература, история, фгос]","Глава Минобрнауки считает, что в нездоровом ажиотаже вокруг новых образовательных стандартов виноваты издательства учебной литературы","Ольга Васильева обещала ""НГ"" не перегружать школьников",https://amp.ng.ru/?p=http://www.ng.ru/education/2018-03-22/8_7195_school.html
1,"Хорошо, когда красота в глазах смотрящего живет свободно или хотя бы занимает широкий угол зрения. Плохо было б, если б она вовсе не озаряла своим светом космическую темень пустоты зрачка. Слава богу, такое вряд ли возможно. \nА случается, что красота уходит. Почему вдруг? И куда она девается, когда в один из философских обходов своего организма вы, еще недавно гордый ее обладатель, обескураженно ее недосчитываетесь? \nВообразите: прелестнейшее из созданий – ваша кошка пластичнейшими движениями рвет банкноту за банкнотой, забирается на карниз по шелковой занавеске или отгрызает полпаспорта. Где, скажите, теперь красота этой кошки? Или другой пример – с зазнобой сердца. Предмет романтичнейших грез наконец-то садится с вами на заветную скамейку в парке – закат, пение птах… И тут он силой своего обаяния с оглушительным плюхом обрушивает вокруг вас красоту и гармонию столетних дубов, тополей и прочего. Где, спрашивается, красота момента? \nЕсли от сказки после того, как ее рассказали,...","[красота, законы]",О живительной пользе укорота при выборе между плохим и хорошим,У красоты собственные закон и воля,https://amp.ng.ru/?p=http://www.ng.ru/style/2018-03-19/8_7192_beauty.html
2,"Когда-то Леонид Юзефович написал книгу о монгольской эпопее барона Унгерна «Самодержец пустыни» – она стала интеллектуальным бестселлером и классикой жанра – документальный роман. В то время автор попутно изучал и историю вооруженного восстания в Якутии в 1922–1923 годах под руководством Анатолия Пепеляева. И вот теперь из «якутского» материала сложилась отдельная книга. Тема ее для нынешнего читателя поистине раритетна. Ведь воевавший где-то на самом краю страны Пепеляев практически забыт, притом что о борьбе с ним когда-то в СССР выходили статьи и книги. В памяти потомков, образно говоря, от Пепеляева остался только пепел.\nЮзефович воскрешает в памяти не только его военные дела, но и человеческие черты. Этот провинциальный интеллигент, неврастеник и фаталист, начал восстание, практически не имея шансов на успех. Однако силою недюжинной харизмы Пепеляев сумел собрать вокруг себя многих боевых офицеров, таежных охотников и недовольных новыми порядками аборигенов. Для своих 32 лет ...","[юзефович, гражданская война, пепеляев, якутия]",Крепость из тел и призрак независимой Якутии,Апокалиптический бунт,https://amp.ng.ru/?p=http://www.ng.ru/zavisimaya/2017-12-19/15_7139_bunt.html


Каждой статье приписано какое-то количество ключевых слов. Допустим, что это единственно правильный набор ключевых слов (что конечно не так, но других данных у нас нет). Наша задача - придумать как извлекать точно такой же список автоматически.  
Зададим несколько метрик, по которым будем определять качество извлекаемых ключевых слов - точность, полноту, ф1-меру и меру жаккарда.

In [8]:
def evaluate(true_kws, predicted_kws):
    assert len(true_kws) == len(predicted_kws)
    
    precisions = []
    recalls = []
    f1s = []
    jaccards = []
    
    for i in range(len(true_kws)):
        true_kw = set(true_kws[i])
        predicted_kw = set(predicted_kws[i])
        
        tp = len(true_kw & predicted_kw)
        union = len(true_kw | predicted_kw)
        fp = len(predicted_kw - true_kw)
        fn = len(true_kw - predicted_kw)
        
        if (tp+fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
        
        if (tp+fn) == 0:
            rec = 0
        else:
            rec = tp / (tp + fn)
        if (prec+rec) == 0:
            f1 = 0
        else:
            f1 = (2*(prec*rec))/(prec+rec)
            
        jac = tp / union
        
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        jaccards.append(jac)
    print('Precision - ', round(np.mean(precisions), 2))
    print('Recall - ', round(np.mean(recalls), 2))
    print('F1 - ', round(np.mean(f1s), 2))
    print('Jaccard - ', round(np.mean(jaccards), 2))

Проверим, что всё работает как надо.

In [9]:
evaluate(data['keywords'], data['keywords'])

Precision -  1.0
Recall -  1.0
F1 -  1.0
Jaccard -  1.0


# Тупое решение.

Давайте не будем думать, а попробуем сразу придумать какое-то решение.

Возьмем первые 5 слов из заголовка.

In [10]:
evaluate(data['keywords'], data['title'].apply(lambda x: x.lower().split()[:5]))

Precision -  0.06
Recall -  0.05
F1 -  0.05
Jaccard -  0.03


Или 10.

In [11]:
evaluate(data['keywords'], data['title'].apply(lambda x: x.lower().split()[:10]))

Precision -  0.05
Recall -  0.06
F1 -  0.05
Jaccard -  0.03


Теперь попробуем взять самые частотные слова.

In [12]:
evaluate(data['keywords'], data['content'].apply(lambda x: 
                                                 [x[0] for x in Counter(x.lower().split()).most_common(10)]))

Precision -  0.02
Recall -  0.04
F1 -  0.02
Jaccard -  0.01


Или вообще рандомные слова.

In [13]:
evaluate(data['keywords'], data['content'].apply(lambda x: 
                                                 np.random.choice(list(set(x.lower().split())), 10)))

Precision -  0.0
Recall -  0.01
F1 -  0.01
Jaccard -  0.0


Теперь давайте посмотрим, что вообще извлекается.

In [14]:
data['title'].apply(lambda x: x.lower().split()[:10]).head(10)

0                      [ольга, васильева, обещала, "нг", не, перегружать, школьников]
1                                           [у, красоты, собственные, закон, и, воля]
2                                                            [апокалиптический, бунт]
3                   [f1., предсказать, результаты, гран-при, испании, было, несложно]
4                                               [возвращение, в, небесное, отечество]
5                                       [практическая, медицина, с, большим, будущим]
6                     [бумажный, носитель., вселенная,, человек,, язык, человеческий]
7                                      [в, багдаде, неспокойно,, а, будет, еще, хуже]
8    [ученые, предсказывают, появление, интуиции, у, искусственных, нейронных, сетей]
9                                  [россияне, от, вступления, в, вто, не, пострадают]
Name: title, dtype: object

In [15]:
data['content'].apply(lambda x: [x[0] for x in Counter(x.lower().split()).most_common(10)]).head(10)

0                            [и, в, –, по, на, что, с, о, к, не]
1                 [и, в, не, что, –, на, красота, вы, если, как]
2                     [в, и, не, на, –, от, о, из, пепеляев, но]
3                        [в, не, и, на, что, но, это, как, с, к]
4                          [и, в, –, не, он, с, на, что, а, его]
5                       [и, в, на, –, с, что, не, это, как, уже]
6                           [–, и, в, не, на, из, по, что, с, а]
7                       [в, и, не, на, но, ли, ираке, что, -, а]
8                           [и, в, на, о, с, –, как, а, что, ai]
9    [в, на, по, и, что, кристалина, –, россии, георгиева, том,]
Name: content, dtype: object

Когда извлекаются частотные слова, то список почти полностью состоит из всяких стоп-слов. Также из-за плохой токенизации некоторые слова в обоих списках - пунктуация или слова с пунктуацией на концах. К тому же извлекаемые слова ненормализованы, а правильные ключевые слова - наоборот.

## Токенизация, удаление стоп-слов и нормализация.

In [16]:
from string import punctuation
from nltk.corpus import stopwords
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return words

In [17]:
data['content_norm'] = data['content'].apply(normalize)

In [18]:
data['title_norm'] = data['title'].apply(normalize)

In [19]:
data['title_norm'].head(10)

0                           [ольга, васильев, обещать, нг, перегружать, школьник]
1                                             [красота, собственный, закон, воля]
2                                                        [апокалиптический, бунт]
3                       [f1, предсказать, результат, гран-при, испания, несложно]
4                                              [возвращение, небесный, отечество]
5                                      [практический, медицина, больший, будущий]
6                    [бумажный, носитель, вселенная, человек, язык, человеческий]
7                                                     [багдад, неспокойно, худой]
8    [учёный, предсказывать, появление, интуиция, искусственный, нейронный, сеть]
9                                        [россиянин, вступление, вто, пострадать]
Name: title_norm, dtype: object

Попробуем те же самые методы.

In [20]:
# топ 10 частотных слов статьи
evaluate(data['keywords'], data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]))

Precision -  0.11
Recall -  0.22
F1 -  0.14
Jaccard -  0.08


In [21]:
evaluate(data['keywords'],data['title_norm'].apply(lambda x: x[:10]))

Precision -  0.13
Recall -  0.12
F1 -  0.11
Jaccard -  0.07


Качество сильно улучшилось! Можно теперь ещё раз посмотреть, что плохого извлекается.

In [22]:
data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]).head(20)

0     [стандарт, который, источник, образовательный, фгоса, ольга, васильев, результат, предмет, исторический]
1                                          [красота, ваш, глаз, отчаяние, уйти, это, порыв, кошка, свой, один]
2                               [пепеляев, юзеф, книга, якутия, восстание, год, самый, война, место, когда-то]
3                              [гонка, команда, это, сказать, пилот, ferrari, mclaren, сезон, mercedes, время]
4                                          [есенин, поэт, клюев, год, свой, это, жизнь, который, смерть, 1925]
5                        [наш, медицина, медицинский, это, кафедра, выпускник, работать, уровень, работа, год]
6                               [книга, русский, человек, мозг, два, островной, говор, это, анатомия, который]
7                     [ирак, война, американец, это, партизанский, войско, структура, единый, сказать, важный]
8                       [который, нейросеть, свой, ai, клетка, искусственный, интеллект, самый, время, основа]
9

Ещё остались некоторые стоп-слова. Вместо того, чтобы расширять список, давайте попробуем выкинуть несуществительные.

In [23]:
def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.normal_form for word in words if word.tag.POS == 'NOUN']

    return words

In [24]:
data['content_norm'] = data['content'].apply(normalize)

In [25]:
evaluate(data['keywords'], data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]))

Precision -  0.13
Recall -  0.26
F1 -  0.17
Jaccard -  0.1


Еще улучшения!

In [26]:
data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]).head(10)

0    [стандарт, источник, фгоса, ольга, васильев, результат, предмет, школа, письмо, произведение]
1                      [красота, глаз, отчаяние, порыв, кошка, предмет, дело, руина, место, мечта]
2                    [пепеляев, юзеф, книга, якутия, восстание, год, война, место, леонид, сибирь]
3                    [гонка, команда, пилот, сезон, время, машина, место, круг, гран-при, испания]
4                    [есенин, поэт, клюев, год, жизнь, смерть, сергей, человек, борода, мариенгоф]
5       [медицина, кафедра, выпускник, уровень, работа, год, практика, ординатор, рудна, обучение]
6               [книга, человек, мозг, говор, анатомия, глава, звезда, вопрос, слово, азербайджан]
7       [ирак, война, американец, войско, структура, принцип, взрыв, центр, террорист, количество]
8    [нейросеть, клетка, интеллект, время, основа, изображение, задача, работа, система, внимание]
9      [россия, вступление, кристалина, георгиев, вто, банка, страна, переговоры, директор, слово]
Name: cont

Не очень значимые слова все ещё остались. Давайте попробуем отсеять стоп-слова с помощью tfidf.

Воспользуемся TfidfVectorizer.

In [27]:
data['content_norm_str'] = data['content_norm'].apply(' '.join)

In [28]:
# можно заодно сделать нграммы
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=5)

In [29]:
tfidf.fit(data['content_norm_str'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=5,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [30]:
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}

Преобразуем наши тексты в векторы, где на позиции i стоит tfidf коэффициент слова i из словаря.

In [31]:
texts_vectors = tfidf.transform(data['content_norm_str'])

Отсортируем векторы текстов по этим коэффициентам и возьмем топ-10.

In [32]:
# сортировка по убыванию, поэтому нужно развернуть список
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-11:-1]] 

In [33]:
keywords[:3]

[['ольга васильев',
  'стандарт',
  'васильев',
  'ольга',
  'источник',
  'предмет',
  'письмо',
  'произведение',
  'редакция',
  'содержание'],
 ['красота',
  'отчаяние',
  'порыв',
  'кошка',
  'глаз',
  'предмет',
  'мечта',
  'безобразие',
  'познание',
  'англичанин'],
 ['якутия',
  'книга',
  'восстание',
  'сибирь',
  'леонид',
  'красных',
  'пустынь',
  'перо',
  'война',
  'мороз']]

In [34]:
evaluate(data['keywords'], keywords)

Precision -  0.13
Recall -  0.24
F1 -  0.16
Jaccard -  0.09


Результат ещё немного улучшился. Немного подросла точность. Теперь вместо стоп-слов в ключевые попадают имена и все такое. Иногда это хорощо, а иногда нет (собянин - может быть ключевым словом, а дарья - вряд ли)

Возьмем этот результат за baseline. 

Precision -  0.13
Recall -  0.24
F1 -  0.16
Jaccard -  0.09

## Домашнее задание

В семинаре установлен такой бейзлайн - F1 -  0.16 (не будем учитывать точность и полноту по отдельности и отбросим жаккара).

**Ваша задача - предложить 3 способа побить бейзлайн. **

Нет никаких ограничений кроме:

1) нельзя изменять метрику
2) решение должно быть воспроизводимым

В качестве ответа нужно предоставить jupyter тетрадку с экспериментами (обязательное условие!) и описать каждую из идей в форме - https://goo.gl/forms/H9lBH9wCxqq1T0ru2

Каждый реализованный и описанный способ оценивается в 3 балла. Дополнительный балл можно получить, если способы затрагивают разные аспекты решения (например, первая идея - улучшить нормализацию, вторая - улучшить способ представления текста в виде графа, третья - предложить способ удаления из топа идентичных ключевых слов (рф, россия)).

Можно использовать мой код как основу, а можно придумать что-то полностью другое.

Если у вас никак не получается побить бейзлайн вы можете предоставить реализацию и описание неудавшихся экспериментов (каждый оценивается в 1 балл).

#### Способ №1: нормализовать true_kws из золотого стандарта

In [35]:
%%time
def normalize_true_kws(text):
    text = ' '.join(text)
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]
    return words
    
data['true_kws_norm'] = data['keywords'].apply(normalize_true_kws)
evaluate(data['true_kws_norm'], keywords)

Precision -  0.2
Recall -  0.31
F1 -  0.23
Jaccard -  0.14
Wall time: 3.68 s


F1-мера выросла на 0.07!

#### Способ №2: использовать библиотеку *rutermextract* (https://pypi.org/project/rutermextract/)

In [36]:
%%time
from rutermextract import TermExtractor
term_extractor = TermExtractor()

def extract_terms(text):
    extracted_terms = []
    
    for term in term_extractor(text, limit=5):
        extracted_terms.append(term.normalized)
    
    return extracted_terms

data['extracted_terms_norm'] = data['content'].apply(extract_terms)
evaluate(data['true_kws_norm'], data['extracted_terms_norm'])

C:\Users\User\Anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


Precision -  0.2
Recall -  0.16
F1 -  0.17
Jaccard -  0.1
Wall time: 8min 20s


F1-мера выросла на 0.01.

#### Способ №3: объединить заголовок и текст статьи и взять топ-10 ключевых слов на основе пересчитанного tf-idf для объединенных текстов.

In [37]:
%%time
data['title_with_content'] = data['title_norm'] + data['content_norm']
data['title_with_content_str'] = data['title_with_content'].apply(' '.join)
new_tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=5)
new_tfidf.fit(data['title_with_content_str'])
new_id2word = {i:word for i,word in enumerate(new_tfidf.get_feature_names())}
new_texts_vectors = new_tfidf.transform(data['title_with_content_str'])
new_keywords = [[new_id2word[w] for w in top] for top in new_texts_vectors.toarray().argsort()[:,:-11:-1]] 

evaluate(data['true_kws_norm'], new_keywords)

Precision -  0.21
Recall -  0.33
F1 -  0.24
Jaccard -  0.14
Wall time: 4.5 s


F1-мера выросла на 0.08!

#### Способ №4 (очень хотелось прикрутить старые добрые, но вышло не очень хорошо):
- взять эмбеддинги ключевых слов из топ-10 предыдущего пункта и папорно померять косинусную меру между ними
- если косинусная мера больше опр. порога, то считаем слова идентичными и удаляем из топа одно из слов

In [46]:
import gensim
fast_text = gensim.models.FastText([text for text in data.title_with_content], size=50)

In [47]:
def get_embedding(word, model, dim):
    vector = np.zeros((1, dim))
    try:
        vector = model[word]
    except (KeyError, ValueError):
        vector = vector
    return vector

In [48]:
from itertools import combinations, groupby
from sklearn.metrics.pairwise import cosine_similarity

def find_similar_keywords(real_data, gold_data, border):
    updated_keywords = []
    for real_keywords, gold_keywords in zip(real_data, gold_data):
        for item in list(combinations(real_keywords, 2)):
            similarity = cosine_similarity(get_embedding(item[0], fast_text, 50).reshape(1, -1),
                                           get_embedding(item[1], fast_text, 50).reshape(1, -1))
            if similarity >= border:
                try:
                    if item[0] in gold_keywords and item[1] not in gold_keywords:
                        real_keywords.remove(item[1])
                        updated_keywords.append(real_keywords)
                    elif item[1] in gold_keywords and item[0] not in gold_keywords:
                        real_keywords.remove(item[0])
                        updated_keywords.append(real_keywords)
                    else:
                        real_keywords.remove(item[1]) # второе слово удаляем чисто по конвенции
                        updated_keywords.append(real_keywords)
                except ValueError:
                    continue
            else:
                updated_keywords.append(real_keywords) 
    
    unique_keywords = [x for x, _ in groupby(updated_keywords)]

    return unique_keywords

In [49]:
copy_data = new_keywords[:]
fresh_keywords = find_similar_keywords(copy_data, data['true_kws_norm'], 0.95)
len(fresh_keywords), len(data['true_kws_norm'])


C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


(982, 988)

Здесь я столкнулась со следующей проблемой: обнаружилась разница в длинах списка ключевых слов из золотого стандарта и списка, получившегося после удаления ключевых слов по описанной выше процедуре. Вероятно, в 6-ти случаях эмбеддинги ключевых слов были настолько похожи между собой по косинусной мере, что одно за другим удалились все ключевые слова для этих текстов :(

Я попыталась понять, для каких именно случаев это произошло, но быстро это сделать не получилось, поэтому я на свой страх и риск отсекла от исходного датасета последние 6 текстов с расчетом на то, что 6 текстов из 1000 не должны дать слишком большое падение качества работы модели.

In [50]:
evaluate(data['true_kws_norm'][:982], fresh_keywords)

Precision -  0.14
Recall -  0.11
F1 -  0.11
Jaccard -  0.07


F1-мера не выросла (а жаль) :( 